In [1]:
# !pip3 install -r ../requirements.txt

In [2]:
import sys
import os

# プロジェクトのルートディレクトリを指定
project_root = os.path.abspath(os.path.join(os.getcwd(), './..'))
sys.path.append(project_root)

## YAML

In [3]:
from omegaconf import OmegaConf

config_path = './../config/config.yaml'
config = OmegaConf.load(config_path)


## Map

In [4]:
from src.map import MapManager

map = config.map.name
line = config.map.line
delimiter = ';'
map_manager = MapManager(map_name=map, raceline=line, delimiter=delimiter, speed=6.0 ,dir_path='./../f1tenth_racetracks/')

## 環境

In [ ]:
from f1tenth_gym.f110_env import F110Env
from src.env import F110_Wrapped

env = F110Env(map=map_manager.map_path, map_ext='.png' ,num_agents=1, num_beams = 1080)
env = F110_Wrapped(env=env, map_manager=map_manager)

## planner

In [6]:
from src.planner import PurePursuitPlanner
wheelbase=(0.17145+0.15875)
planner = PurePursuitPlanner(wheelbase=wheelbase, map_manager=map_manager, lookahead=0.3 ,max_reacquire=20.) 

In [7]:
import numpy as np

obs = env.reset()
done = False
# シミュレーションループ
while not done:
    actions = []
    steer, speed = planner.plan(obs, gain=0.20)
    
    action = [steer, speed]
    actions.append(action)
    obs, step_reward, done, info = env.step(np.array(actions))
    